In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix,ConfusionMatrixDisplay,roc_curve,roc_auc_score,precision_recall_curve
from sklearn.ensemble import RandomForestClassifier , StackingClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,Lasso
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_val_score,GridSearchCV

# 한글 깨짐 방지
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'

In [2]:
kk = pd.read_csv('./datasets/KOSDAQ_DATA.csv', index_col=0)
kk = kk.iloc[:,2:]

In [3]:
kk[kk['회계년도'] == '2011/12']['label'].value_counts()

0    788
1     31
Name: label, dtype: int64

In [4]:
kk[kk['회계년도'] == '2012/12']['label'].value_counts()

0    805
1     21
Name: label, dtype: int64

In [5]:
kk[kk['회계년도'] == '2013/12']['label'].value_counts()

0    845
1     14
Name: label, dtype: int64

In [6]:
kk[kk['회계년도'] == '2014/12']['label'].value_counts()

0    891
1     13
Name: label, dtype: int64

In [7]:
kk[kk['회계년도'] == '2015/12']['label'].value_counts()

0    944
1     10
Name: label, dtype: int64

In [8]:
kk[kk['회계년도'] == '2016/12']['label'].value_counts()

0    1008
1       7
Name: label, dtype: int64

In [9]:
kk[kk['회계년도'] == '2017/12']['label'].value_counts()

0    1075
1      11
Name: label, dtype: int64

In [10]:
kk[kk['회계년도'] == '2018/12']['label'].value_counts()

0    1157
1       6
Name: label, dtype: int64

In [11]:
kk[kk['회계년도'] == '2019/12']['label'].value_counts()

0    1230
1      12
Name: label, dtype: int64

In [12]:
train_df = pd.read_csv('./datasets/KOSDAQ+KOSPI_TRAIN_DATA.csv', index_col=0)
train_df = train_df.iloc[:,2:]
test_df = pd.read_csv('./datasets/KOSDAQ+KOSPI_TEST1_DATA.csv', index_col=0)
test_df = test_df.iloc[:,2:]

# 코스닥 + 코스피

In [13]:
# 교차검증 1 데이터 셋
kk_1112 = kk[(kk['회계년도'] <= '2012/12') & (kk['회계년도'] >= '2011/12')]
kk_1319 = kk[(kk['회계년도'] <= '2019/12') & (kk['회계년도'] >= '2013/12')]
# 교차검증 2 데이터 셋
kk_1213 = kk[(kk['회계년도'] <= '2013/12') & (kk['회계년도'] >= '2012/12')]
kk_111419 = kk[(kk['회계년도'] <= '2019/12') & (kk['회계년도'] >= '2014/12') | (kk['회계년도'] == '2011/12')]
# 교차검증 3 데이터 셋
kk_1314 = kk[(kk['회계년도'] <= '2014/12') & (kk['회계년도'] >= '2013/12')]
kk_11121519 = kk[(kk['회계년도'] <= '2019/12') & (kk['회계년도'] >= '2015/12') | (kk['회계년도'] >= '2011/12') & (kk['회계년도'] <= '2012/12')]
# 교차검증 4 데이터 셋
kk_1415 = kk[(kk['회계년도'] <= '2015/12') & (kk['회계년도'] >= '2014/12')]
kk_11131619 = kk[(kk['회계년도'] <= '2019/12') & (kk['회계년도'] >= '2016/12') | (kk['회계년도'] >= '2011/12') & (kk['회계년도'] <= '2013/12')]
# 교차검증 5 데이터 셋
kk_1516 = kk[(kk['회계년도'] <= '2016/12') & (kk['회계년도'] >= '2015/12')]
kk_11141719 = kk[(kk['회계년도'] <= '2019/12') & (kk['회계년도'] >= '2017/12') | (kk['회계년도'] >= '2011/12') & (kk['회계년도'] <= '2014/12')]
# 교차검증 6 데이터 셋
kk_1617 = kk[(kk['회계년도'] <= '2017/12') & (kk['회계년도'] >= '2016/12')]
kk_11151819 = kk[(kk['회계년도'] <= '2019/12') & (kk['회계년도'] >= '2018/12') | (kk['회계년도'] >= '2011/12') & (kk['회계년도'] <= '2015/12')]
# 교차검증 7 데이터 셋
kk_1718 = kk[(kk['회계년도'] <= '2018/12') & (kk['회계년도'] >= '2017/12')]
kk_111619 = kk[(kk['회계년도'] == '2019/12') | (kk['회계년도'] >= '2011/12') & (kk['회계년도'] <= '2016/12')]

In [14]:
kk_1718['label'].value_counts()

0    2232
1      17
Name: label, dtype: int64

# 코스닥

In [15]:
# # 교차검증 1 데이터 셋
# kosdaq_1112 = kosdaq[(kosdaq['회계년도'] <= '2012/12') & (kosdaq['회계년도'] >= '2011/12')]
# kosdaq_1319 = kosdaq[(kosdaq['회계년도'] <= '2019/12') & (kosdaq['회계년도'] >= '2013/12')]
# # 교차검증 2 데이터 셋
# kosdaq_1213 = kosdaq[(kosdaq['회계년도'] <= '2013/12') & (kosdaq['회계년도'] >= '2012/12')]
# kosdaq_111419 = kosdaq[(kosdaq['회계년도'] <= '2019/12') & (kosdaq['회계년도'] >= '2014/12') | (kosdaq['회계년도'] == '2011/12')]
# # 교차검증 3 데이터 셋
# kosdaq_1314 = kosdaq[(kosdaq['회계년도'] <= '2014/12') & (kosdaq['회계년도'] >= '2013/12')]
# kosdaq_11121519 = kosdaq[(kosdaq['회계년도'] <= '2019/12') & (kosdaq['회계년도'] >= '2015/12') | (kosdaq['회계년도'] >= '2011/12') & (kosdaq['회계년도'] <= '2012/12')]
# # 교차검증 4 데이터 셋
# kosdaq_1415 = kosdaq[(kosdaq['회계년도'] <= '2015/12') & (kosdaq['회계년도'] >= '2014/12')]
# kosdaq_11131619 = kosdaq[(kosdaq['회계년도'] <= '2019/12') & (kosdaq['회계년도'] >= '2016/12') | (kosdaq['회계년도'] >= '2011/12') & (kosdaq['회계년도'] <= '2013/12')]
# # 교차검증 5 데이터 셋
# kosdaq_1516 = kosdaq[(kosdaq['회계년도'] <= '2016/12') & (kosdaq['회계년도'] >= '2015/12')]
# kosdaq_11141719 = kosdaq[(kosdaq['회계년도'] <= '2019/12') & (kosdaq['회계년도'] >= '2017/12') | (kosdaq['회계년도'] >= '2011/12') & (kosdaq['회계년도'] <= '2014/12')]
# # 교차검증 6 데이터 셋
# kosdaq_1617 = kosdaq[(kosdaq['회계년도'] <= '2017/12') & (kosdaq['회계년도'] >= '2016/12')]
# kosdaq_11151819 = kosdaq[(kosdaq['회계년도'] <= '2019/12') & (kosdaq['회계년도'] >= '2018/12') | (kosdaq['회계년도'] >= '2011/12') & (kosdaq['회계년도'] <= '2015/12')]
# # 교차검증 7 데이터 셋
# kosdaq_1718 = kosdaq[(kosdaq['회계년도'] <= '2018/12') & (kosdaq['회계년도'] >= '2017/12')]
# kosdaq_111619 = kosdaq[(kosdaq['회계년도'] == '2019/12') | (kosdaq['회계년도'] >= '2011/12') & (kosdaq['회계년도'] <= '2016/12')]

In [16]:
kk[(kk['회계년도'] <= '2012/12') & (kk['회계년도'] >= '2011/12')]

,회계년도,총자본증가율,유형자산증가율,유동자산증가율,비유동자산증가율,재고자산증가율,자기자본증가율,매출액증가율,매출총이익증가율,당기순이익증가율,...,현금및현금성자산의증가,경영기간,상장기간,label,산업분류,기업생애주기_Shake-Out,기업생애주기_도입기,기업생애주기_성숙기,기업생애주기_성장기,기업생애주기_쇠퇴기
0,2011/12,1.975650,-5.902879,17.636684,-12.887724,-1.437298,-2.015757,56.405286,137.025215,78.286537,...,-1295.0,10.0,11.0,0,1,0,1,0,0,0
1,2012/12,109.148508,-4.130614,207.079069,-16.424273,7.728764,75.174670,-8.432853,-37.961217,-341.644562,...,3522.0,11.0,12.0,0,1,0,1,0,0,0
11,2011/12,12.783346,27.822222,3.829865,30.736804,27.477802,-7.918632,21.630551,36.474359,-193.872549,...,-1321.0,34.0,10.0,0,0,0,0,0,1,0
12,2012/12,-16.574436,-26.240148,-33.367569,10.168699,-8.810241,1.564491,4.853878,5.220425,92.514595,...,911.0,35.0,11.0,0,0,0,0,0,1,0
22,2011/12,22.758542,11.463942,23.694934,22.016892,-54.838710,30.731209,30.071634,25.350919,25.874753,...,2231.0,12.0,7.0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11516,2012/12,14.596670,26.034978,11.606015,21.244369,8.694872,18.330311,17.588384,13.057595,7.981221,...,4537.0,32.0,11.0,0,1,0,0,0,1,0
11534,2011/12,35.071544,27.830619,38.252846,29.382820,17.697084,37.531486,58.705540,73.252321,6.774194,...,1933.0,13.0,6.0,0,1,0,1,0,0,0
11535,2012/12,-8.497146,-1.253695,-13.744924,1.537542,-5.929579,-2.551893,-38.142627,-49.680826,-487.160121,...,-1056.0,14.0,7.0,0,1,0,1,0,0,0
11556,2011/12,-1.973172,-5.854894,-5.497614,-0.978158,-24.431818,2.087027,8.116816,8.881911,13.451631,...,514.0,45.0,18.0,0,0,1,0,0,0,0


In [17]:
kk[(kk['회계년도'] <= '2019/12') & (kk['회계년도'] >= '2013/12')]['label'].value_counts()

0    7150
1      73
Name: label, dtype: int64

# 검증 함수 만들기

# 1번 데이터 검증

In [18]:
def model1(Data):

    train = Data[(Data['회계년도'] <= '2019/12') & (Data['회계년도'] >= '2013/12')].reset_index(drop=True)
    test = Data[(Data['회계년도'] <= '2012/12') & (Data['회계년도'] >= '2011/12')].reset_index(drop=True)

    ## Train Scaling 전 Data 분리하기 ##

    X_train = train.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_train_concat = train[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_train = train[['label']]
    X_test = test.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_test_concat = test[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_test = test[['label']]


    ## Scaling##

    from sklearn.preprocessing import StandardScaler
    # StandardScaler() Scaler객체 생성.
    scaler = StandardScaler()
    # 학습 데이터에 대해서 fit(), transform() 수행.
    scaler.fit(X_train)
    train_scaled_SS = scaler.transform(X_train)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_train_SS = pd.DataFrame(data = train_scaled_SS, columns=X_train.columns)

    # 테스트 데이터에서는 다시 fit(), transform()이나 fit_transform()을 수행하지 않고 transform만 수행.
    test_scaled_MM = scaler.transform(X_test)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_test_SS = pd.DataFrame(data = test_scaled_MM, columns=X_test.columns)


    X_train_SS = pd.concat([X_train_SS,X_train_concat,y_train], axis=1)
    X_test_SS = pd.concat([X_test_SS,X_test_concat, y_test], axis=1)

    trian_test_df = pd.concat([X_train_SS, X_test_SS], axis=0)
    trian_test_df.reset_index(drop=True, inplace=True)      

    df_2011 = trian_test_df[trian_test_df['회계년도'] == '2011/12']
    df_2012 = trian_test_df[trian_test_df['회계년도'] == '2012/12']
    df_2013 = trian_test_df[trian_test_df['회계년도'] == '2013/12']
    df_2014 = trian_test_df[trian_test_df['회계년도'] == '2014/12']
    df_2015 = trian_test_df[trian_test_df['회계년도'] == '2015/12']
    df_2016 = trian_test_df[trian_test_df['회계년도'] == '2016/12']
    df_2017 = trian_test_df[trian_test_df['회계년도'] == '2017/12']
    df_2018 = trian_test_df[trian_test_df['회계년도'] == '2018/12']
    df_2019 = trian_test_df[trian_test_df['회계년도'] == '2019/12']

    df_X_2011 = df_2011.drop(columns='label', axis=1)
    df_y_2011 = df_2011[['label']]
    df_X_2012 = df_2012.drop(columns='label', axis=1)
    df_y_2012 = df_2012[['label']]
    df_X_2013 = df_2013.drop(columns='label', axis=1)
    df_y_2013 = df_2013[['label']]
    df_X_2014 = df_2014.drop(columns='label', axis=1)
    df_y_2014 = df_2014[['label']]
    df_X_2015 = df_2015.drop(columns='label', axis=1)
    df_y_2015 = df_2015[['label']]
    df_X_2016 = df_2016.drop(columns='label', axis=1)
    df_y_2016 = df_2016[['label']]
    df_X_2017 = df_2017.drop(columns='label', axis=1)
    df_y_2017 = df_2017[['label']]
    df_X_2018 = df_2018.drop(columns='label', axis=1)
    df_y_2018 = df_2018[['label']]
    df_X_2019 = df_2019.drop(columns='label', axis=1)
    df_y_2019 = df_2019[['label']]
    

    ## UnderSampler ##

    from imblearn.under_sampling import RandomUnderSampler

    X_samp_11, y_samp_11 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2011, df_y_2011)
    X_samp_12, y_samp_12 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2012, df_y_2012)
    X_samp_13, y_samp_13 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2013, df_y_2013)
    X_samp_14, y_samp_14 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2014, df_y_2014)
    X_samp_15, y_samp_15 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2015, df_y_2015)
    X_samp_16, y_samp_16 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2016, df_y_2016)
    X_samp_17, y_samp_17 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2017, df_y_2017)
    X_samp_18, y_samp_18 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2018, df_y_2018)
    X_samp_19, y_samp_19 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2019, df_y_2019)

    X_samp = pd.concat([X_samp_11, X_samp_12, X_samp_13, X_samp_14, X_samp_15, X_samp_16, X_samp_17, X_samp_18, X_samp_19], axis=0)
    y_samp = pd.concat([y_samp_11, y_samp_12, y_samp_13, y_samp_14, y_samp_15, y_samp_16, y_samp_17, y_samp_18, y_samp_19], axis=0)

    trian_test_samp_df = pd.concat([X_samp, y_samp], axis=1)

    # 교차검증 1 데이터 셋
    X_samp_train = trian_test_samp_df[(trian_test_samp_df['회계년도'] <= '2019/12') & (trian_test_samp_df['회계년도'] >= '2013/12')]
    X_samp_test = trian_test_samp_df[(trian_test_samp_df['회계년도'] <= '2012/12') & (trian_test_samp_df['회계년도'] >= '2011/12')]

    X_samp_train_1 = X_samp_train.drop(columns=['label', '회계년도'], axis=1)
    y_samp_train_1 = X_samp_train[['label']]

    X_samp_test_1 = X_samp_test.drop(columns=['label', '회계년도'], axis=1)
    y_samp_test_1 = X_samp_test[['label']]

    X_samp_train_1.reset_index(drop=True, inplace=True)
    X_samp_test_1.reset_index(drop=True, inplace=True)
    y_samp_train_1.reset_index(drop=True, inplace=True)
    y_samp_test_1.reset_index(drop=True, inplace=True)

    from imblearn.over_sampling import BorderlineSMOTE

    X_samp_2, y_samp_2 = BorderlineSMOTE(random_state=0,sampling_strategy=1).fit_resample(X_samp_train_1, y_samp_train_1)

    train_df = pd.concat([X_samp_2, y_samp_2], axis=1)
    test_df = pd.concat([X_samp_test_1, y_samp_test_1], axis=1)

    return train_df, test_df

In [19]:
model1(kk)[0]['label'].value_counts()

0    365
1    365
Name: label, dtype: int64

In [20]:
model1(kk)[1]['label'].value_counts()

0    260
1     52
Name: label, dtype: int64

# 2번 데이터 검증

In [21]:
def model2(Data):

    train = Data[(Data['회계년도'] <= '2019/12') & (Data['회계년도'] >= '2014/12') | (Data['회계년도'] == '2011/12')].reset_index(drop=True)
    test = Data[(Data['회계년도'] <= '2013/12') & (Data['회계년도'] >= '2012/12')].reset_index(drop=True)
    ## Train Scaling 전 Data 분리하기 ##

    X_train = train.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_train_concat = train[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_train = train[['label']]
    X_test = test.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_test_concat = test[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_test = test[['label']]


    ## Scaling##

    from sklearn.preprocessing import StandardScaler
    # StandardScaler() Scaler객체 생성.
    scaler = StandardScaler()
    # 학습 데이터에 대해서 fit(), transform() 수행.
    scaler.fit(X_train)
    train_scaled_SS = scaler.transform(X_train)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_train_SS = pd.DataFrame(data = train_scaled_SS, columns=X_train.columns)

    # 테스트 데이터에서는 다시 fit(), transform()이나 fit_transform()을 수행하지 않고 transform만 수행.
    test_scaled_MM = scaler.transform(X_test)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_test_SS = pd.DataFrame(data = test_scaled_MM, columns=X_test.columns)


    X_train_SS = pd.concat([X_train_SS,X_train_concat,y_train], axis=1)
    X_test_SS = pd.concat([X_test_SS,X_test_concat, y_test], axis=1)

    trian_test_df = pd.concat([X_train_SS, X_test_SS], axis=0)
    trian_test_df.reset_index(drop=True, inplace=True)      

    df_2011 = trian_test_df[trian_test_df['회계년도'] == '2011/12']
    df_2012 = trian_test_df[trian_test_df['회계년도'] == '2012/12']
    df_2013 = trian_test_df[trian_test_df['회계년도'] == '2013/12']
    df_2014 = trian_test_df[trian_test_df['회계년도'] == '2014/12']
    df_2015 = trian_test_df[trian_test_df['회계년도'] == '2015/12']
    df_2016 = trian_test_df[trian_test_df['회계년도'] == '2016/12']
    df_2017 = trian_test_df[trian_test_df['회계년도'] == '2017/12']
    df_2018 = trian_test_df[trian_test_df['회계년도'] == '2018/12']
    df_2019 = trian_test_df[trian_test_df['회계년도'] == '2019/12']

    df_X_2011 = df_2011.drop(columns='label', axis=1)
    df_y_2011 = df_2011[['label']]
    df_X_2012 = df_2012.drop(columns='label', axis=1)
    df_y_2012 = df_2012[['label']]
    df_X_2013 = df_2013.drop(columns='label', axis=1)
    df_y_2013 = df_2013[['label']]
    df_X_2014 = df_2014.drop(columns='label', axis=1)
    df_y_2014 = df_2014[['label']]
    df_X_2015 = df_2015.drop(columns='label', axis=1)
    df_y_2015 = df_2015[['label']]
    df_X_2016 = df_2016.drop(columns='label', axis=1)
    df_y_2016 = df_2016[['label']]
    df_X_2017 = df_2017.drop(columns='label', axis=1)
    df_y_2017 = df_2017[['label']]
    df_X_2018 = df_2018.drop(columns='label', axis=1)
    df_y_2018 = df_2018[['label']]
    df_X_2019 = df_2019.drop(columns='label', axis=1)
    df_y_2019 = df_2019[['label']]
    

    ## UnderSampler ##

    from imblearn.under_sampling import RandomUnderSampler

    X_samp_11, y_samp_11 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2011, df_y_2011)
    X_samp_12, y_samp_12 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2012, df_y_2012)
    X_samp_13, y_samp_13 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2013, df_y_2013)
    X_samp_14, y_samp_14 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2014, df_y_2014)
    X_samp_15, y_samp_15 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2015, df_y_2015)
    X_samp_16, y_samp_16 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2016, df_y_2016)
    X_samp_17, y_samp_17 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2017, df_y_2017)
    X_samp_18, y_samp_18 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2018, df_y_2018)
    X_samp_19, y_samp_19 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2019, df_y_2019)

    X_samp = pd.concat([X_samp_11, X_samp_12, X_samp_13, X_samp_14, X_samp_15, X_samp_16, X_samp_17, X_samp_18, X_samp_19], axis=0)
    y_samp = pd.concat([y_samp_11, y_samp_12, y_samp_13, y_samp_14, y_samp_15, y_samp_16, y_samp_17, y_samp_18, y_samp_19], axis=0)

    a = pd.concat([X_samp, y_samp], axis=1)

    # 교차검증 2 데이터 셋
    X_samp_train = a[(a['회계년도'] <= '2019/12') & (a['회계년도'] >= '2014/12') | (a['회계년도'] == '2011/12')]
    X_samp_test = a[(a['회계년도'] <= '2013/12') & (a['회계년도'] >= '2012/12')]

    X_samp_train_1 = X_samp_train.drop(columns=['label', '회계년도'], axis=1)
    y_samp_train_1 = X_samp_train[['label']]

    X_samp_test_1 = X_samp_test.drop(columns=['label', '회계년도'], axis=1)
    y_samp_test_1 = X_samp_test[['label']]

    X_samp_train_1.reset_index(drop=True, inplace=True)
    X_samp_test_1.reset_index(drop=True, inplace=True)
    y_samp_train_1.reset_index(drop=True, inplace=True)
    y_samp_test_1.reset_index(drop=True, inplace=True)

    from imblearn.over_sampling import BorderlineSMOTE

    X_samp_2, y_samp_2 = BorderlineSMOTE(random_state=0,sampling_strategy=1).fit_resample(X_samp_train_1, y_samp_train_1)

    train_df = pd.concat([X_samp_2, y_samp_2], axis=1)
    test_df = pd.concat([X_samp_test_1, y_samp_test_1], axis=1)

    return train_df, test_df

In [22]:
model2(kk)[0]['label'].value_counts()

0    450
1    450
Name: label, dtype: int64

In [23]:
model2(kk)[1]['label'].value_counts()

0    175
1     35
Name: label, dtype: int64

# 3번 데이터 검증

In [24]:
def model3(Data):

    train = Data[(Data['회계년도'] <= '2019/12') & (Data['회계년도'] >= '2015/12') | (Data['회계년도'] >= '2011/12') & (Data['회계년도'] <= '2012/12')].reset_index(drop=True)
    test = Data[(Data['회계년도'] <= '2014/12') & (Data['회계년도'] >= '2013/12')].reset_index(drop=True)

    X_train = train.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_train_concat = train[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_train = train[['label']]
    X_test = test.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_test_concat = test[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_test = test[['label']]


    ## Scaling##

    from sklearn.preprocessing import StandardScaler
    # StandardScaler() Scaler객체 생성.
    scaler = StandardScaler()
    # 학습 데이터에 대해서 fit(), transform() 수행.
    scaler.fit(X_train)
    train_scaled_SS = scaler.transform(X_train)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_train_SS = pd.DataFrame(data = train_scaled_SS, columns=X_train.columns)

    # 테스트 데이터에서는 다시 fit(), transform()이나 fit_transform()을 수행하지 않고 transform만 수행.
    test_scaled_MM = scaler.transform(X_test)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_test_SS = pd.DataFrame(data = test_scaled_MM, columns=X_test.columns)


    X_train_SS = pd.concat([X_train_SS,X_train_concat,y_train], axis=1)
    X_test_SS = pd.concat([X_test_SS,X_test_concat, y_test], axis=1)

    trian_test_df = pd.concat([X_train_SS, X_test_SS], axis=0)
    trian_test_df.reset_index(drop=True, inplace=True)      

    df_2011 = trian_test_df[trian_test_df['회계년도'] == '2011/12']
    df_2012 = trian_test_df[trian_test_df['회계년도'] == '2012/12']
    df_2013 = trian_test_df[trian_test_df['회계년도'] == '2013/12']
    df_2014 = trian_test_df[trian_test_df['회계년도'] == '2014/12']
    df_2015 = trian_test_df[trian_test_df['회계년도'] == '2015/12']
    df_2016 = trian_test_df[trian_test_df['회계년도'] == '2016/12']
    df_2017 = trian_test_df[trian_test_df['회계년도'] == '2017/12']
    df_2018 = trian_test_df[trian_test_df['회계년도'] == '2018/12']
    df_2019 = trian_test_df[trian_test_df['회계년도'] == '2019/12']

    df_X_2011 = df_2011.drop(columns='label', axis=1)
    df_y_2011 = df_2011[['label']]
    df_X_2012 = df_2012.drop(columns='label', axis=1)
    df_y_2012 = df_2012[['label']]
    df_X_2013 = df_2013.drop(columns='label', axis=1)
    df_y_2013 = df_2013[['label']]
    df_X_2014 = df_2014.drop(columns='label', axis=1)
    df_y_2014 = df_2014[['label']]
    df_X_2015 = df_2015.drop(columns='label', axis=1)
    df_y_2015 = df_2015[['label']]
    df_X_2016 = df_2016.drop(columns='label', axis=1)
    df_y_2016 = df_2016[['label']]
    df_X_2017 = df_2017.drop(columns='label', axis=1)
    df_y_2017 = df_2017[['label']]
    df_X_2018 = df_2018.drop(columns='label', axis=1)
    df_y_2018 = df_2018[['label']]
    df_X_2019 = df_2019.drop(columns='label', axis=1)
    df_y_2019 = df_2019[['label']]
    

    ## UnderSampler ##

    from imblearn.under_sampling import RandomUnderSampler

    X_samp_11, y_samp_11 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2011, df_y_2011)
    X_samp_12, y_samp_12 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2012, df_y_2012)
    X_samp_13, y_samp_13 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2013, df_y_2013)
    X_samp_14, y_samp_14 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2014, df_y_2014)
    X_samp_15, y_samp_15 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2015, df_y_2015)
    X_samp_16, y_samp_16 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2016, df_y_2016)
    X_samp_17, y_samp_17 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2017, df_y_2017)
    X_samp_18, y_samp_18 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2018, df_y_2018)
    X_samp_19, y_samp_19 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2019, df_y_2019)

    X_samp = pd.concat([X_samp_11, X_samp_12, X_samp_13, X_samp_14, X_samp_15, X_samp_16, X_samp_17, X_samp_18, X_samp_19], axis=0)
    y_samp = pd.concat([y_samp_11, y_samp_12, y_samp_13, y_samp_14, y_samp_15, y_samp_16, y_samp_17, y_samp_18, y_samp_19], axis=0)

    a = pd.concat([X_samp, y_samp], axis=1)

    # 교차검증 3 데이터 셋
    X_samp_train = a[(a['회계년도'] <= '2019/12') & (a['회계년도'] >= '2015/12') | (a['회계년도'] >= '2011/12') & (a['회계년도'] <= '2012/12')]
    X_samp_test = a[(a['회계년도'] <= '2014/12') & (a['회계년도'] >= '2013/12')]

    X_samp_train_1 = X_samp_train.drop(columns=['label', '회계년도'], axis=1)
    y_samp_train_1 = X_samp_train[['label']]

    X_samp_test_1 = X_samp_test.drop(columns=['label', '회계년도'], axis=1)
    y_samp_test_1 = X_samp_test[['label']]

    X_samp_train_1.reset_index(drop=True, inplace=True)
    X_samp_test_1.reset_index(drop=True, inplace=True)
    y_samp_train_1.reset_index(drop=True, inplace=True)
    y_samp_test_1.reset_index(drop=True, inplace=True)

    from imblearn.over_sampling import BorderlineSMOTE

    X_samp_2, y_samp_2 = BorderlineSMOTE(random_state=0,sampling_strategy=1).fit_resample(X_samp_train_1, y_samp_train_1)

    train_df = pd.concat([X_samp_2, y_samp_2], axis=1)
    test_df = pd.concat([X_samp_test_1, y_samp_test_1], axis=1)

    return train_df, test_df

In [25]:
model3(kk)[0]['label'].value_counts()

0    490
1    490
Name: label, dtype: int64

In [26]:
model3(kk)[1]['label'].value_counts()

0    135
1     27
Name: label, dtype: int64

# 4번 데이터 검증

In [27]:
def model4(Data):

    train = Data[(Data['회계년도'] <= '2019/12') & (Data['회계년도'] >= '2016/12') | (Data['회계년도'] >= '2011/12') & (Data['회계년도'] <= '2013/12')].reset_index(drop=True)
    test = Data[(Data['회계년도'] <= '2015/12') & (Data['회계년도'] >= '2014/12')].reset_index(drop=True)

    X_train = train.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_train_concat = train[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_train = train[['label']]
    X_test = test.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_test_concat = test[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_test = test[['label']]


    ## Scaling##

    from sklearn.preprocessing import StandardScaler
    # StandardScaler() Scaler객체 생성.
    scaler = StandardScaler()
    # 학습 데이터에 대해서 fit(), transform() 수행.
    scaler.fit(X_train)
    train_scaled_SS = scaler.transform(X_train)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_train_SS = pd.DataFrame(data = train_scaled_SS, columns=X_train.columns)

    # 테스트 데이터에서는 다시 fit(), transform()이나 fit_transform()을 수행하지 않고 transform만 수행.
    test_scaled_MM = scaler.transform(X_test)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_test_SS = pd.DataFrame(data = test_scaled_MM, columns=X_test.columns)


    X_train_SS = pd.concat([X_train_SS,X_train_concat,y_train], axis=1)
    X_test_SS = pd.concat([X_test_SS,X_test_concat, y_test], axis=1)

    trian_test_df = pd.concat([X_train_SS, X_test_SS], axis=0)
    trian_test_df.reset_index(drop=True, inplace=True)      

    df_2011 = trian_test_df[trian_test_df['회계년도'] == '2011/12']
    df_2012 = trian_test_df[trian_test_df['회계년도'] == '2012/12']
    df_2013 = trian_test_df[trian_test_df['회계년도'] == '2013/12']
    df_2014 = trian_test_df[trian_test_df['회계년도'] == '2014/12']
    df_2015 = trian_test_df[trian_test_df['회계년도'] == '2015/12']
    df_2016 = trian_test_df[trian_test_df['회계년도'] == '2016/12']
    df_2017 = trian_test_df[trian_test_df['회계년도'] == '2017/12']
    df_2018 = trian_test_df[trian_test_df['회계년도'] == '2018/12']
    df_2019 = trian_test_df[trian_test_df['회계년도'] == '2019/12']

    df_X_2011 = df_2011.drop(columns='label', axis=1)
    df_y_2011 = df_2011[['label']]
    df_X_2012 = df_2012.drop(columns='label', axis=1)
    df_y_2012 = df_2012[['label']]
    df_X_2013 = df_2013.drop(columns='label', axis=1)
    df_y_2013 = df_2013[['label']]
    df_X_2014 = df_2014.drop(columns='label', axis=1)
    df_y_2014 = df_2014[['label']]
    df_X_2015 = df_2015.drop(columns='label', axis=1)
    df_y_2015 = df_2015[['label']]
    df_X_2016 = df_2016.drop(columns='label', axis=1)
    df_y_2016 = df_2016[['label']]
    df_X_2017 = df_2017.drop(columns='label', axis=1)
    df_y_2017 = df_2017[['label']]
    df_X_2018 = df_2018.drop(columns='label', axis=1)
    df_y_2018 = df_2018[['label']]
    df_X_2019 = df_2019.drop(columns='label', axis=1)
    df_y_2019 = df_2019[['label']]
    

    ## UnderSampler ##

    from imblearn.under_sampling import RandomUnderSampler

    X_samp_11, y_samp_11 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2011, df_y_2011)
    X_samp_12, y_samp_12 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2012, df_y_2012)
    X_samp_13, y_samp_13 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2013, df_y_2013)
    X_samp_14, y_samp_14 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2014, df_y_2014)
    X_samp_15, y_samp_15 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2015, df_y_2015)
    X_samp_16, y_samp_16 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2016, df_y_2016)
    X_samp_17, y_samp_17 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2017, df_y_2017)
    X_samp_18, y_samp_18 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2018, df_y_2018)
    X_samp_19, y_samp_19 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2019, df_y_2019)

    X_samp = pd.concat([X_samp_11, X_samp_12, X_samp_13, X_samp_14, X_samp_15, X_samp_16, X_samp_17, X_samp_18, X_samp_19], axis=0)
    y_samp = pd.concat([y_samp_11, y_samp_12, y_samp_13, y_samp_14, y_samp_15, y_samp_16, y_samp_17, y_samp_18, y_samp_19], axis=0)

    a = pd.concat([X_samp, y_samp], axis=1)

    # 교차검증 3 데이터 셋
    X_samp_train = a[(a['회계년도'] <= '2019/12') & (a['회계년도'] >= '2016/12') | (a['회계년도'] >= '2011/12') & (a['회계년도'] <= '2013/12')]
    X_samp_test = a[(a['회계년도'] <= '2015/12') & (a['회계년도'] >= '2014/12')]

    X_samp_train_1 = X_samp_train.drop(columns=['label', '회계년도'], axis=1)
    y_samp_train_1 = X_samp_train[['label']]

    X_samp_test_1 = X_samp_test.drop(columns=['label', '회계년도'], axis=1)
    y_samp_test_1 = X_samp_test[['label']]

    X_samp_train_1.reset_index(drop=True, inplace=True)
    X_samp_test_1.reset_index(drop=True, inplace=True)
    y_samp_train_1.reset_index(drop=True, inplace=True)
    y_samp_test_1.reset_index(drop=True, inplace=True)

    from imblearn.over_sampling import BorderlineSMOTE

    X_samp_2, y_samp_2 = BorderlineSMOTE(random_state=0,sampling_strategy=1).fit_resample(X_samp_train_1, y_samp_train_1)

    train_df = pd.concat([X_samp_2, y_samp_2], axis=1)
    test_df = pd.concat([X_samp_test_1, y_samp_test_1], axis=1)

    return train_df, test_df

In [28]:
model4(kk)[0]['label'].value_counts()

0    510
1    510
Name: label, dtype: int64

In [29]:
model4(kk)[1]['label'].value_counts()

0    115
1     23
Name: label, dtype: int64

# 5번 데이터 검증

In [30]:
def model5(Data):

    train = Data[(Data['회계년도'] <= '2019/12') & (Data['회계년도'] >= '2017/12') | (Data['회계년도'] >= '2011/12') & (Data['회계년도'] <= '2014/12')].reset_index(drop=True)
    test = Data[(Data['회계년도'] <= '2016/12') & (Data['회계년도'] >= '2015/12')].reset_index(drop=True)

    X_train = train.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_train_concat = train[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_train = train[['label']]
    X_test = test.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_test_concat = test[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_test = test[['label']]


    ## Scaling##

    from sklearn.preprocessing import StandardScaler
    # StandardScaler() Scaler객체 생성.
    scaler = StandardScaler()
    # 학습 데이터에 대해서 fit(), transform() 수행.
    scaler.fit(X_train)
    train_scaled_SS = scaler.transform(X_train)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_train_SS = pd.DataFrame(data = train_scaled_SS, columns=X_train.columns)

    # 테스트 데이터에서는 다시 fit(), transform()이나 fit_transform()을 수행하지 않고 transform만 수행.
    test_scaled_MM = scaler.transform(X_test)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_test_SS = pd.DataFrame(data = test_scaled_MM, columns=X_test.columns)


    X_train_SS = pd.concat([X_train_SS,X_train_concat,y_train], axis=1)
    X_test_SS = pd.concat([X_test_SS,X_test_concat, y_test], axis=1)

    trian_test_df = pd.concat([X_train_SS, X_test_SS], axis=0)
    trian_test_df.reset_index(drop=True, inplace=True)      

    df_2011 = trian_test_df[trian_test_df['회계년도'] == '2011/12']
    df_2012 = trian_test_df[trian_test_df['회계년도'] == '2012/12']
    df_2013 = trian_test_df[trian_test_df['회계년도'] == '2013/12']
    df_2014 = trian_test_df[trian_test_df['회계년도'] == '2014/12']
    df_2015 = trian_test_df[trian_test_df['회계년도'] == '2015/12']
    df_2016 = trian_test_df[trian_test_df['회계년도'] == '2016/12']
    df_2017 = trian_test_df[trian_test_df['회계년도'] == '2017/12']
    df_2018 = trian_test_df[trian_test_df['회계년도'] == '2018/12']
    df_2019 = trian_test_df[trian_test_df['회계년도'] == '2019/12']

    df_X_2011 = df_2011.drop(columns='label', axis=1)
    df_y_2011 = df_2011[['label']]
    df_X_2012 = df_2012.drop(columns='label', axis=1)
    df_y_2012 = df_2012[['label']]
    df_X_2013 = df_2013.drop(columns='label', axis=1)
    df_y_2013 = df_2013[['label']]
    df_X_2014 = df_2014.drop(columns='label', axis=1)
    df_y_2014 = df_2014[['label']]
    df_X_2015 = df_2015.drop(columns='label', axis=1)
    df_y_2015 = df_2015[['label']]
    df_X_2016 = df_2016.drop(columns='label', axis=1)
    df_y_2016 = df_2016[['label']]
    df_X_2017 = df_2017.drop(columns='label', axis=1)
    df_y_2017 = df_2017[['label']]
    df_X_2018 = df_2018.drop(columns='label', axis=1)
    df_y_2018 = df_2018[['label']]
    df_X_2019 = df_2019.drop(columns='label', axis=1)
    df_y_2019 = df_2019[['label']]
    

    ## UnderSampler ##

    from imblearn.under_sampling import RandomUnderSampler

    X_samp_11, y_samp_11 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2011, df_y_2011)
    X_samp_12, y_samp_12 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2012, df_y_2012)
    X_samp_13, y_samp_13 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2013, df_y_2013)
    X_samp_14, y_samp_14 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2014, df_y_2014)
    X_samp_15, y_samp_15 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2015, df_y_2015)
    X_samp_16, y_samp_16 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2016, df_y_2016)
    X_samp_17, y_samp_17 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2017, df_y_2017)
    X_samp_18, y_samp_18 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2018, df_y_2018)
    X_samp_19, y_samp_19 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2019, df_y_2019)

    X_samp = pd.concat([X_samp_11, X_samp_12, X_samp_13, X_samp_14, X_samp_15, X_samp_16, X_samp_17, X_samp_18, X_samp_19], axis=0)
    y_samp = pd.concat([y_samp_11, y_samp_12, y_samp_13, y_samp_14, y_samp_15, y_samp_16, y_samp_17, y_samp_18, y_samp_19], axis=0)

    a = pd.concat([X_samp, y_samp], axis=1)

    # 교차검증 3 데이터 셋
    X_samp_train = a[(a['회계년도'] <= '2019/12') & (a['회계년도'] >= '2017/12') | (a['회계년도'] >= '2011/12') & (a['회계년도'] <= '2014/12')]
    X_samp_test = a[(a['회계년도'] <= '2016/12') & (a['회계년도'] >= '2015/12')]

    X_samp_train_1 = X_samp_train.drop(columns=['label', '회계년도'], axis=1)
    y_samp_train_1 = X_samp_train[['label']]

    X_samp_test_1 = X_samp_test.drop(columns=['label', '회계년도'], axis=1)
    y_samp_test_1 = X_samp_test[['label']]

    X_samp_train_1.reset_index(drop=True, inplace=True)
    X_samp_test_1.reset_index(drop=True, inplace=True)
    y_samp_train_1.reset_index(drop=True, inplace=True)
    y_samp_test_1.reset_index(drop=True, inplace=True)

    from imblearn.over_sampling import BorderlineSMOTE

    X_samp_2, y_samp_2 = BorderlineSMOTE(random_state=0,sampling_strategy=1).fit_resample(X_samp_train_1, y_samp_train_1)

    train_df = pd.concat([X_samp_2, y_samp_2], axis=1)
    test_df = pd.concat([X_samp_test_1, y_samp_test_1], axis=1)

    return train_df, test_df

In [31]:
model5(kk)[0]['label'].value_counts()

0    540
1    540
Name: label, dtype: int64

In [32]:
model5(kk)[1]['label'].value_counts()

0    85
1    17
Name: label, dtype: int64

# 6번 데이터 검증

In [33]:
def model6(Data):

    train = Data[(Data['회계년도'] <= '2019/12') & (Data['회계년도'] >= '2018/12') | (Data['회계년도'] >= '2011/12') & (Data['회계년도'] <= '2015/12')].reset_index(drop=True)
    test = Data[(Data['회계년도'] <= '2017/12') & (Data['회계년도'] >= '2016/12')].reset_index(drop=True)

    X_train = train.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_train_concat = train[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_train = train[['label']]
    X_test = test.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_test_concat = test[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_test = test[['label']]


    ## Scaling##

    from sklearn.preprocessing import StandardScaler
    # StandardScaler() Scaler객체 생성.
    scaler = StandardScaler()
    # 학습 데이터에 대해서 fit(), transform() 수행.
    scaler.fit(X_train)
    train_scaled_SS = scaler.transform(X_train)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_train_SS = pd.DataFrame(data = train_scaled_SS, columns=X_train.columns)

    # 테스트 데이터에서는 다시 fit(), transform()이나 fit_transform()을 수행하지 않고 transform만 수행.
    test_scaled_MM = scaler.transform(X_test)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_test_SS = pd.DataFrame(data = test_scaled_MM, columns=X_test.columns)


    X_train_SS = pd.concat([X_train_SS,X_train_concat,y_train], axis=1)
    X_test_SS = pd.concat([X_test_SS,X_test_concat, y_test], axis=1)

    trian_test_df = pd.concat([X_train_SS, X_test_SS], axis=0)
    trian_test_df.reset_index(drop=True, inplace=True)      

    df_2011 = trian_test_df[trian_test_df['회계년도'] == '2011/12']
    df_2012 = trian_test_df[trian_test_df['회계년도'] == '2012/12']
    df_2013 = trian_test_df[trian_test_df['회계년도'] == '2013/12']
    df_2014 = trian_test_df[trian_test_df['회계년도'] == '2014/12']
    df_2015 = trian_test_df[trian_test_df['회계년도'] == '2015/12']
    df_2016 = trian_test_df[trian_test_df['회계년도'] == '2016/12']
    df_2017 = trian_test_df[trian_test_df['회계년도'] == '2017/12']
    df_2018 = trian_test_df[trian_test_df['회계년도'] == '2018/12']
    df_2019 = trian_test_df[trian_test_df['회계년도'] == '2019/12']

    df_X_2011 = df_2011.drop(columns='label', axis=1)
    df_y_2011 = df_2011[['label']]
    df_X_2012 = df_2012.drop(columns='label', axis=1)
    df_y_2012 = df_2012[['label']]
    df_X_2013 = df_2013.drop(columns='label', axis=1)
    df_y_2013 = df_2013[['label']]
    df_X_2014 = df_2014.drop(columns='label', axis=1)
    df_y_2014 = df_2014[['label']]
    df_X_2015 = df_2015.drop(columns='label', axis=1)
    df_y_2015 = df_2015[['label']]
    df_X_2016 = df_2016.drop(columns='label', axis=1)
    df_y_2016 = df_2016[['label']]
    df_X_2017 = df_2017.drop(columns='label', axis=1)
    df_y_2017 = df_2017[['label']]
    df_X_2018 = df_2018.drop(columns='label', axis=1)
    df_y_2018 = df_2018[['label']]
    df_X_2019 = df_2019.drop(columns='label', axis=1)
    df_y_2019 = df_2019[['label']]
    

    ## UnderSampler ##

    from imblearn.under_sampling import RandomUnderSampler

    X_samp_11, y_samp_11 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2011, df_y_2011)
    X_samp_12, y_samp_12 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2012, df_y_2012)
    X_samp_13, y_samp_13 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2013, df_y_2013)
    X_samp_14, y_samp_14 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2014, df_y_2014)
    X_samp_15, y_samp_15 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2015, df_y_2015)
    X_samp_16, y_samp_16 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2016, df_y_2016)
    X_samp_17, y_samp_17 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2017, df_y_2017)
    X_samp_18, y_samp_18 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2018, df_y_2018)
    X_samp_19, y_samp_19 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2019, df_y_2019)

    X_samp = pd.concat([X_samp_11, X_samp_12, X_samp_13, X_samp_14, X_samp_15, X_samp_16, X_samp_17, X_samp_18, X_samp_19], axis=0)
    y_samp = pd.concat([y_samp_11, y_samp_12, y_samp_13, y_samp_14, y_samp_15, y_samp_16, y_samp_17, y_samp_18, y_samp_19], axis=0)

    a = pd.concat([X_samp, y_samp], axis=1)

    # 교차검증 3 데이터 셋
    X_samp_train = a[(a['회계년도'] <= '2019/12') & (a['회계년도'] >= '2018/12') | (a['회계년도'] >= '2011/12') & (a['회계년도'] <= '2015/12')]
    X_samp_test = a[(a['회계년도'] <= '2017/12') & (a['회계년도'] >= '2016/12')]

    X_samp_train_1 = X_samp_train.drop(columns=['label', '회계년도'], axis=1)
    y_samp_train_1 = X_samp_train[['label']]

    X_samp_test_1 = X_samp_test.drop(columns=['label', '회계년도'], axis=1)
    y_samp_test_1 = X_samp_test[['label']]

    X_samp_train_1.reset_index(drop=True, inplace=True)
    X_samp_test_1.reset_index(drop=True, inplace=True)
    y_samp_train_1.reset_index(drop=True, inplace=True)
    y_samp_test_1.reset_index(drop=True, inplace=True)

    from imblearn.over_sampling import BorderlineSMOTE

    X_samp_2, y_samp_2 = BorderlineSMOTE(random_state=0,sampling_strategy=1).fit_resample(X_samp_train_1, y_samp_train_1)

    train_df = pd.concat([X_samp_2, y_samp_2], axis=1)
    test_df = pd.concat([X_samp_test_1, y_samp_test_1], axis=1)

    return train_df, test_df

In [34]:
model6(kk)[0]['label'].value_counts()

0    535
1    535
Name: label, dtype: int64

In [35]:
model6(kk)[1]['label'].value_counts()

0    90
1    18
Name: label, dtype: int64

# 7번 데이터 검증

In [36]:
def model7(Data):

    train = Data[(Data['회계년도'] == '2019/12') | (Data['회계년도'] >= '2011/12') & (Data['회계년도'] <= '2016/12')].reset_index(drop=True)
    test = Data[(Data['회계년도'] <= '2018/12') & (Data['회계년도'] >= '2017/12')].reset_index(drop=True)

    X_train = train.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_train_concat = train[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_train = train[['label']]
    X_test = test.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_test_concat = test[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_test = test[['label']]


    ## Scaling##

    from sklearn.preprocessing import StandardScaler
    # StandardScaler() Scaler객체 생성.
    scaler = StandardScaler()
    # 학습 데이터에 대해서 fit(), transform() 수행.
    scaler.fit(X_train)
    train_scaled_SS = scaler.transform(X_train)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_train_SS = pd.DataFrame(data = train_scaled_SS, columns=X_train.columns)

    # 테스트 데이터에서는 다시 fit(), transform()이나 fit_transform()을 수행하지 않고 transform만 수행.
    test_scaled_MM = scaler.transform(X_test)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_test_SS = pd.DataFrame(data = test_scaled_MM, columns=X_test.columns)


    X_train_SS = pd.concat([X_train_SS,X_train_concat,y_train], axis=1)
    X_test_SS = pd.concat([X_test_SS,X_test_concat, y_test], axis=1)

    trian_test_df = pd.concat([X_train_SS, X_test_SS], axis=0)
    trian_test_df.reset_index(drop=True, inplace=True)      

    df_2011 = trian_test_df[trian_test_df['회계년도'] == '2011/12']
    df_2012 = trian_test_df[trian_test_df['회계년도'] == '2012/12']
    df_2013 = trian_test_df[trian_test_df['회계년도'] == '2013/12']
    df_2014 = trian_test_df[trian_test_df['회계년도'] == '2014/12']
    df_2015 = trian_test_df[trian_test_df['회계년도'] == '2015/12']
    df_2016 = trian_test_df[trian_test_df['회계년도'] == '2016/12']
    df_2017 = trian_test_df[trian_test_df['회계년도'] == '2017/12']
    df_2018 = trian_test_df[trian_test_df['회계년도'] == '2018/12']
    df_2019 = trian_test_df[trian_test_df['회계년도'] == '2019/12']

    df_X_2011 = df_2011.drop(columns='label', axis=1)
    df_y_2011 = df_2011[['label']]
    df_X_2012 = df_2012.drop(columns='label', axis=1)
    df_y_2012 = df_2012[['label']]
    df_X_2013 = df_2013.drop(columns='label', axis=1)
    df_y_2013 = df_2013[['label']]
    df_X_2014 = df_2014.drop(columns='label', axis=1)
    df_y_2014 = df_2014[['label']]
    df_X_2015 = df_2015.drop(columns='label', axis=1)
    df_y_2015 = df_2015[['label']]
    df_X_2016 = df_2016.drop(columns='label', axis=1)
    df_y_2016 = df_2016[['label']]
    df_X_2017 = df_2017.drop(columns='label', axis=1)
    df_y_2017 = df_2017[['label']]
    df_X_2018 = df_2018.drop(columns='label', axis=1)
    df_y_2018 = df_2018[['label']]
    df_X_2019 = df_2019.drop(columns='label', axis=1)
    df_y_2019 = df_2019[['label']]
    

    ## UnderSampler ##

    from imblearn.under_sampling import RandomUnderSampler

    X_samp_11, y_samp_11 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2011, df_y_2011)
    X_samp_12, y_samp_12 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2012, df_y_2012)
    X_samp_13, y_samp_13 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2013, df_y_2013)
    X_samp_14, y_samp_14 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2014, df_y_2014)
    X_samp_15, y_samp_15 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2015, df_y_2015)
    X_samp_16, y_samp_16 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2016, df_y_2016)
    X_samp_17, y_samp_17 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2017, df_y_2017)
    X_samp_18, y_samp_18 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2018, df_y_2018)
    X_samp_19, y_samp_19 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2019, df_y_2019)

    X_samp = pd.concat([X_samp_11, X_samp_12, X_samp_13, X_samp_14, X_samp_15, X_samp_16, X_samp_17, X_samp_18, X_samp_19], axis=0)
    y_samp = pd.concat([y_samp_11, y_samp_12, y_samp_13, y_samp_14, y_samp_15, y_samp_16, y_samp_17, y_samp_18, y_samp_19], axis=0)

    a = pd.concat([X_samp, y_samp], axis=1)

    # 교차검증 3 데이터 셋
    X_samp_train = a[(a['회계년도'] == '2019/12') | (a['회계년도'] >= '2011/12') & (a['회계년도'] <= '2016/12')]
    X_samp_test = a[(a['회계년도'] <= '2018/12') & (a['회계년도'] >= '2017/12')]

    X_samp_train_1 = X_samp_train.drop(columns=['label', '회계년도'], axis=1)
    y_samp_train_1 = X_samp_train[['label']]

    X_samp_test_1 = X_samp_test.drop(columns=['label', '회계년도'], axis=1)
    y_samp_test_1 = X_samp_test[['label']]

    X_samp_train_1.reset_index(drop=True, inplace=True)
    X_samp_test_1.reset_index(drop=True, inplace=True)
    y_samp_train_1.reset_index(drop=True, inplace=True)
    y_samp_test_1.reset_index(drop=True, inplace=True)

    from imblearn.over_sampling import BorderlineSMOTE

    X_samp_2, y_samp_2 = BorderlineSMOTE(random_state=0,sampling_strategy=1).fit_resample(X_samp_train_1, y_samp_train_1)

    train_df = pd.concat([X_samp_2, y_samp_2], axis=1)
    test_df = pd.concat([X_samp_test_1, y_samp_test_1], axis=1)

    return train_df, test_df

In [37]:
model7(kk)[0]['label'].value_counts()

0    540
1    540
Name: label, dtype: int64

In [38]:
model7(kk)[1]['label'].value_counts()

0    85
1    17
Name: label, dtype: int64

# 0번 데이터 검증

In [39]:
def model0(Data):

    train = Data[(Data['회계년도'] <= '2017/12') & (Data['회계년도'] >= '2011/12')].reset_index(drop=True)
    test = Data[(Data['회계년도'] <= '2019/12') & (Data['회계년도'] >= '2018/12')].reset_index(drop=True)

    ## Train Scaling 전 Data 분리하기 ##

    X_train = train.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_train_concat = train[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_train = train[['label']]
    X_test = test.drop(columns=['label', '산업분류', '기업생애주기_Shake-Out',
       '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도'], axis=1)
    X_test_concat = test[['산업분류', '기업생애주기_Shake-Out',
        '기업생애주기_도입기', '기업생애주기_성숙기', '기업생애주기_성장기', '기업생애주기_쇠퇴기', '회계년도']]
    y_test = test[['label']]


    ## Scaling##

    from sklearn.preprocessing import StandardScaler
    # StandardScaler() Scaler객체 생성.
    scaler = StandardScaler()
    # 학습 데이터에 대해서 fit(), transform() 수행.
    scaler.fit(X_train)
    train_scaled_SS = scaler.transform(X_train)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_train_SS = pd.DataFrame(data = train_scaled_SS, columns=X_train.columns)

    # 테스트 데이터에서는 다시 fit(), transform()이나 fit_transform()을 수행하지 않고 transform만 수행.
    test_scaled_MM = scaler.transform(X_test)
    # transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
    X_test_SS = pd.DataFrame(data = test_scaled_MM, columns=X_test.columns)


    X_train_SS = pd.concat([X_train_SS,X_train_concat,y_train], axis=1)
    X_test_SS = pd.concat([X_test_SS,X_test_concat, y_test], axis=1)

    trian_test_df = pd.concat([X_train_SS, X_test_SS], axis=0)
    trian_test_df.reset_index(drop=True, inplace=True)      

    df_2011 = trian_test_df[trian_test_df['회계년도'] == '2011/12']
    df_2012 = trian_test_df[trian_test_df['회계년도'] == '2012/12']
    df_2013 = trian_test_df[trian_test_df['회계년도'] == '2013/12']
    df_2014 = trian_test_df[trian_test_df['회계년도'] == '2014/12']
    df_2015 = trian_test_df[trian_test_df['회계년도'] == '2015/12']
    df_2016 = trian_test_df[trian_test_df['회계년도'] == '2016/12']
    df_2017 = trian_test_df[trian_test_df['회계년도'] == '2017/12']
    df_2018 = trian_test_df[trian_test_df['회계년도'] == '2018/12']
    df_2019 = trian_test_df[trian_test_df['회계년도'] == '2019/12']

    df_X_2011 = df_2011.drop(columns='label', axis=1)
    df_y_2011 = df_2011[['label']]
    df_X_2012 = df_2012.drop(columns='label', axis=1)
    df_y_2012 = df_2012[['label']]
    df_X_2013 = df_2013.drop(columns='label', axis=1)
    df_y_2013 = df_2013[['label']]
    df_X_2014 = df_2014.drop(columns='label', axis=1)
    df_y_2014 = df_2014[['label']]
    df_X_2015 = df_2015.drop(columns='label', axis=1)
    df_y_2015 = df_2015[['label']]
    df_X_2016 = df_2016.drop(columns='label', axis=1)
    df_y_2016 = df_2016[['label']]
    df_X_2017 = df_2017.drop(columns='label', axis=1)
    df_y_2017 = df_2017[['label']]
    df_X_2018 = df_2018.drop(columns='label', axis=1)
    df_y_2018 = df_2018[['label']]
    df_X_2019 = df_2019.drop(columns='label', axis=1)
    df_y_2019 = df_2019[['label']]
    

    ## UnderSampler ##

    from imblearn.under_sampling import RandomUnderSampler

    X_samp_11, y_samp_11 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2011, df_y_2011)
    X_samp_12, y_samp_12 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2012, df_y_2012)
    X_samp_13, y_samp_13 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2013, df_y_2013)
    X_samp_14, y_samp_14 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2014, df_y_2014)
    X_samp_15, y_samp_15 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2015, df_y_2015)
    X_samp_16, y_samp_16 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2016, df_y_2016)
    X_samp_17, y_samp_17 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2017, df_y_2017)
    X_samp_18, y_samp_18 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2018, df_y_2018)
    X_samp_19, y_samp_19 = RandomUnderSampler(random_state=0,sampling_strategy=0.2).fit_resample(df_X_2019, df_y_2019)

    X_samp = pd.concat([X_samp_11, X_samp_12, X_samp_13, X_samp_14, X_samp_15, X_samp_16, X_samp_17, X_samp_18, X_samp_19], axis=0)
    y_samp = pd.concat([y_samp_11, y_samp_12, y_samp_13, y_samp_14, y_samp_15, y_samp_16, y_samp_17, y_samp_18, y_samp_19], axis=0)

    trian_test_samp_df = pd.concat([X_samp, y_samp], axis=1)

    # 교차검증 1 데이터 셋
    X_samp_train = trian_test_samp_df[(trian_test_samp_df['회계년도'] <= '2017/12') & (trian_test_samp_df['회계년도'] >= '2011/12')]
    X_samp_test = trian_test_samp_df[(trian_test_samp_df['회계년도'] <= '2019/12') & (trian_test_samp_df['회계년도'] >= '2018/12')]

    X_samp_train_1 = X_samp_train.drop(columns=['label', '회계년도'], axis=1)
    y_samp_train_1 = X_samp_train[['label']]

    X_samp_test_1 = X_samp_test.drop(columns=['label', '회계년도'], axis=1)
    y_samp_test_1 = X_samp_test[['label']]

    X_samp_train_1.reset_index(drop=True, inplace=True)
    X_samp_test_1.reset_index(drop=True, inplace=True)
    y_samp_train_1.reset_index(drop=True, inplace=True)
    y_samp_test_1.reset_index(drop=True, inplace=True)

    from imblearn.over_sampling import BorderlineSMOTE

    X_samp_2, y_samp_2 = BorderlineSMOTE(random_state=0,sampling_strategy=1).fit_resample(X_samp_train_1, y_samp_train_1)

    train_df = pd.concat([X_samp_2, y_samp_2], axis=1)
    test_df = pd.concat([X_samp_test_1, y_samp_test_1], axis=1)

    return train_df, test_df

In [40]:
model0(kk)[0]['label'].value_counts()

0    535
1    535
Name: label, dtype: int64

In [41]:
model0(kk)[1]['label'].value_counts()

0    90
1    18
Name: label, dtype: int64

In [42]:
X_train_1 = model1(kk)[0].drop(columns=['label'], axis=1)
y_train_1 = model1(kk)[0][['label']]
X_test_1 = model1(kk)[1].drop(columns=['label'], axis=1)
y_test_1 = model1(kk)[1][['label']]

In [43]:
X_train_2 = model2(kk)[0].drop(columns=['label'], axis=1)
y_train_2 = model2(kk)[0][['label']]
X_test_2 = model2(kk)[1].drop(columns=['label'], axis=1)
y_test_2 = model2(kk)[1][['label']]

In [44]:
X_train_3 = model3(kk)[0].drop(columns=['label'], axis=1)
y_train_3 = model3(kk)[0][['label']]
X_test_3 = model3(kk)[1].drop(columns=['label'], axis=1)
y_test_3 = model3(kk)[1][['label']]

In [45]:
X_train_4 = model4(kk)[0].drop(columns=['label'], axis=1)
y_train_4 = model4(kk)[0][['label']]
X_test_4 = model4(kk)[1].drop(columns=['label'], axis=1)
y_test_4 = model4(kk)[1][['label']]

In [46]:
X_train_5 = model5(kk)[0].drop(columns=['label'], axis=1)
y_train_5 = model5(kk)[0][['label']]
X_test_5 = model5(kk)[1].drop(columns=['label'], axis=1)
y_test_5 = model5(kk)[1][['label']]

In [47]:
X_train_6 = model6(kk)[0].drop(columns=['label'], axis=1)
y_train_6 = model6(kk)[0][['label']]
X_test_6 = model6(kk)[1].drop(columns=['label'], axis=1)
y_test_6 = model6(kk)[1][['label']]

In [48]:
X_train_7 = model7(kk)[0].drop(columns=['label'], axis=1)
y_train_7 = model7(kk)[0][['label']]
X_test_7 = model7(kk)[1].drop(columns=['label'], axis=1)
y_test_7 = model7(kk)[1][['label']]

In [49]:
def model_basic(x_train, y_train, x_test, y_test): 
    models = [
        LogisticRegression(random_state=0),
        SVC(random_state=0),
        DecisionTreeClassifier(random_state=0),
        RandomForestClassifier(random_state=0),
        XGBClassifier(random_state=0),
        LGBMClassifier(random_state=0) 
    ]

    rdict={'model':[],'acc_train':[], 'auc_train':[], 'acc_test':[],'precision':[],'recall':[],'f1_score':[], 'AUC_test':[]}


    for clf in models:
        clf = clf.fit(x_train, y_train)
    #1열:Train
        y_hat = clf.predict(x_train)
        results_train  = (round(accuracy_score(y_train,y_hat),2),round(roc_auc_score(y_train,y_hat),2))
    #2열:Test
        y_hat = clf.predict(x_test)
        results = (round(accuracy_score(y_test,y_hat),2),
                        round(precision_score(y_test,y_hat),2),
                        round(recall_score(y_test,y_hat),2),
                        round(f1_score(y_test,y_hat),2),
                        round(roc_auc_score(y_test,y_hat),2))

        rdict['model'].append(clf); 
        rdict['acc_train'].append(results_train[0])
        rdict['auc_train'].append(results_train[1])
        
        rdict['acc_test'].append(results[0])
        rdict['precision'].append(results[1])
        rdict['recall'].append(results[2])
        rdict['f1_score'].append(results[3])
        rdict['AUC_test'].append(results[4])   

        # confusion = confusion_matrix(y_test, y_hat)

        # print(confusion)

    rdf_final = pd.DataFrame(data=rdict)
    return rdf_final

In [50]:
a_0 = model_basic(X_train_1, y_train_1, X_test_1, y_test_1).iloc[0]
a_1 = model_basic(X_train_1, y_train_1, X_test_1, y_test_1).iloc[1]
a_2 = model_basic(X_train_1, y_train_1, X_test_1, y_test_1).iloc[2]
a_3 = model_basic(X_train_1, y_train_1, X_test_1, y_test_1).iloc[3]
a_4 = model_basic(X_train_1, y_train_1, X_test_1, y_test_1).iloc[4]
a_5 = model_basic(X_train_1, y_train_1, X_test_1, y_test_1).iloc[5]

In [51]:
b_0 = model_basic(X_train_2, y_train_2, X_test_2, y_test_2).iloc[0]
b_1 = model_basic(X_train_2, y_train_2, X_test_2, y_test_2).iloc[1]
b_2 = model_basic(X_train_2, y_train_2, X_test_2, y_test_2).iloc[2]
b_3 = model_basic(X_train_2, y_train_2, X_test_2, y_test_2).iloc[3]
b_4 = model_basic(X_train_2, y_train_2, X_test_2, y_test_2).iloc[4]
b_5 = model_basic(X_train_2, y_train_2, X_test_2, y_test_2).iloc[5]

In [52]:
c_0 = model_basic(X_train_3, y_train_3, X_test_3, y_test_3).iloc[0]
c_1 = model_basic(X_train_3, y_train_3, X_test_3, y_test_3).iloc[1]
c_2 = model_basic(X_train_3, y_train_3, X_test_3, y_test_3).iloc[2]
c_3 = model_basic(X_train_3, y_train_3, X_test_3, y_test_3).iloc[3]
c_4 = model_basic(X_train_3, y_train_3, X_test_3, y_test_3).iloc[4]
c_5 = model_basic(X_train_3, y_train_3, X_test_3, y_test_3).iloc[5]

In [53]:
d_0 = model_basic(X_train_4, y_train_4, X_test_4, y_test_4).iloc[0]
d_1 = model_basic(X_train_4, y_train_4, X_test_4, y_test_4).iloc[1]
d_2 = model_basic(X_train_4, y_train_4, X_test_4, y_test_4).iloc[2]
d_3 = model_basic(X_train_4, y_train_4, X_test_4, y_test_4).iloc[3]
d_4 = model_basic(X_train_4, y_train_4, X_test_4, y_test_4).iloc[4]
d_5 = model_basic(X_train_4, y_train_4, X_test_4, y_test_4).iloc[5]

In [54]:
e_0 = model_basic(X_train_5, y_train_5, X_test_5, y_test_5).iloc[0]
e_1 = model_basic(X_train_5, y_train_5, X_test_5, y_test_5).iloc[1]
e_2 = model_basic(X_train_5, y_train_5, X_test_5, y_test_5).iloc[2]
e_3 = model_basic(X_train_5, y_train_5, X_test_5, y_test_5).iloc[3]
e_4 = model_basic(X_train_5, y_train_5, X_test_5, y_test_5).iloc[4]
e_5 = model_basic(X_train_5, y_train_5, X_test_5, y_test_5).iloc[5]

In [55]:
f_0 = model_basic(X_train_6, y_train_6, X_test_6, y_test_6).iloc[0]
f_1 = model_basic(X_train_6, y_train_6, X_test_6, y_test_6).iloc[1]
f_2 = model_basic(X_train_6, y_train_6, X_test_6, y_test_6).iloc[2]
f_3 = model_basic(X_train_6, y_train_6, X_test_6, y_test_6).iloc[3]
f_4 = model_basic(X_train_6, y_train_6, X_test_6, y_test_6).iloc[4]
f_5 = model_basic(X_train_6, y_train_6, X_test_6, y_test_6).iloc[5]

In [56]:
g_0 = model_basic(X_train_7, y_train_7, X_test_7, y_test_7).iloc[0]
g_1 = model_basic(X_train_7, y_train_7, X_test_7, y_test_7).iloc[1]
g_2 = model_basic(X_train_7, y_train_7, X_test_7, y_test_7).iloc[2]
g_3 = model_basic(X_train_7, y_train_7, X_test_7, y_test_7).iloc[3]
g_4 = model_basic(X_train_7, y_train_7, X_test_7, y_test_7).iloc[4]
g_5 = model_basic(X_train_7, y_train_7, X_test_7, y_test_7).iloc[5]

In [57]:
df_0 = pd.DataFrame(a_0)
df_1 = pd.DataFrame(b_0)
df_2 = pd.DataFrame(c_0)
df_3 = pd.DataFrame(d_0)
df_4 = pd.DataFrame(e_0)
df_5 = pd.DataFrame(f_0)
df_6 = pd.DataFrame(e_0)

df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6], axis=1)
# 첫 번째 행을 컬럼으로 옮기기
df.columns = df.iloc[0]
df = df.iloc[1:]
dfdf1 = pd.DataFrame(df.T.mean())
dfdf1.columns = ['Logistic']
dfdf1

,Logistic
acc_train,0.968571
auc_train,0.968571
acc_test,0.910000
precision,0.714286
recall,0.790000
f1_score,0.750000
AUC_test,0.861429


In [58]:
df_0 = pd.DataFrame(a_1)
df_1 = pd.DataFrame(b_1)
df_2 = pd.DataFrame(c_1)
df_3 = pd.DataFrame(d_1)
df_4 = pd.DataFrame(e_1)
df_5 = pd.DataFrame(f_1)
df_6 = pd.DataFrame(e_1)

df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6], axis=1)
# 첫 번째 행을 컬럼으로 옮기기
df.columns = df.iloc[0]
df = df.iloc[1:]
dfdf2 = pd.DataFrame(df.T.mean())
dfdf2.columns = ['SVC']
dfdf2

,SVC
acc_train,0.972857
auc_train,0.972857
acc_test,0.910000
precision,0.724286
recall,0.762857
f1_score,0.742857
AUC_test,0.851429


In [59]:
df_0 = pd.DataFrame(a_2)
df_1 = pd.DataFrame(b_2)
df_2 = pd.DataFrame(c_2)
df_3 = pd.DataFrame(d_2)
df_4 = pd.DataFrame(e_2)
df_5 = pd.DataFrame(f_2)
df_6 = pd.DataFrame(e_2)

df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6], axis=1)
# 첫 번째 행을 컬럼으로 옮기기
df.columns = df.iloc[0]
df = df.iloc[1:]
dfdf3 = pd.DataFrame(df.T.mean())
dfdf3.columns = ['DecisionTree']
dfdf3

,DecisionTree
acc_train,1.000000
auc_train,1.000000
acc_test,0.890000
precision,0.647143
recall,0.768571
f1_score,0.700000
AUC_test,0.841429


In [60]:
df_0 = pd.DataFrame(a_3)
df_1 = pd.DataFrame(b_3)
df_2 = pd.DataFrame(c_3)
df_3 = pd.DataFrame(d_3)
df_4 = pd.DataFrame(e_3)
df_5 = pd.DataFrame(f_3)
df_6 = pd.DataFrame(e_3)

df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6], axis=1)
# 첫 번째 행을 컬럼으로 옮기기
df.columns = df.iloc[0]
df = df.iloc[1:]
dfdf4 = pd.DataFrame(df.T.mean())
dfdf4.columns = ['RandomForest']
dfdf4

,RandomForest
acc_train,1.000000
auc_train,1.000000
acc_test,0.924286
precision,0.742857
recall,0.850000
f1_score,0.791429
AUC_test,0.897143


In [61]:
df_0 = pd.DataFrame(a_4)
df_1 = pd.DataFrame(b_4)
df_2 = pd.DataFrame(c_4)
df_3 = pd.DataFrame(d_4)
df_4 = pd.DataFrame(e_4)
df_5 = pd.DataFrame(f_4)
df_6 = pd.DataFrame(e_4)

df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6], axis=1)
# 첫 번째 행을 컬럼으로 옮기기
df.columns = df.iloc[0]
df = df.iloc[1:]
dfdf5 = pd.DataFrame(df.T.mean())
dfdf5.columns = ['XGB']
dfdf5

,XGB
acc_train,1.000000
auc_train,1.000000
acc_test,0.918571
precision,0.731429
recall,0.821429
f1_score,0.772857
AUC_test,0.880000


In [62]:
df_0 = pd.DataFrame(a_5)
df_1 = pd.DataFrame(b_5)
df_2 = pd.DataFrame(c_5)
df_3 = pd.DataFrame(d_5)
df_4 = pd.DataFrame(e_5)
df_5 = pd.DataFrame(f_5)
df_6 = pd.DataFrame(e_5)

df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6], axis=1)
# 첫 번째 행을 컬럼으로 옮기기
df.columns = df.iloc[0]
df = df.iloc[1:]
dfdf6 = pd.DataFrame(df.T.mean())
dfdf6.columns = ['LGBM']
dfdf6

,LGBM
acc_train,1.000000
auc_train,1.000000
acc_test,0.925714
precision,0.757143
recall,0.821429
f1_score,0.788571
AUC_test,0.884286


In [63]:
kosdaqkospi_교차검증평균 = round(pd.concat([dfdf1,dfdf2,dfdf3,dfdf4,dfdf5,dfdf6], axis=1),4)
kosdaqkospi_교차검증평균.T

,acc_train,auc_train,acc_test,precision,recall,f1_score,AUC_test
Logistic,0.9686,0.9686,0.9100,0.7143,0.7900,0.7500,0.8614
SVC,0.9729,0.9729,0.9100,0.7243,0.7629,0.7429,0.8514
DecisionTree,1.0000,1.0000,0.8900,0.6471,0.7686,0.7000,0.8414
RandomForest,1.0000,1.0000,0.9243,0.7429,0.8500,0.7914,0.8971
XGB,1.0000,1.0000,0.9186,0.7314,0.8214,0.7729,0.8800
LGBM,1.0000,1.0000,0.9257,0.7571,0.8214,0.7886,0.8843
